# Out-of-State-Contributions: States Analysis

How much out-of-state money have candidates in each of the states raised in the 2018 election cycle, in absolute and proportional terms, thus far and how does that compare with this point in the 2014 and 2010 cycles?

In [1]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

Import contributions data.

In [2]:
contributions = pd.read_csv("data/contributions.csv")
contributions.info()

/home/jyerardi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,17,18,19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6947770 entries, 0 to 6947769
Data columns (total 23 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      object
contributor_street        object
contributor_city          object
contributor_state         object
contributor_zip           float64
in_out_state              object
no_veto                   object
office                    object
last_day                  object
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: float64(2), int64(2), object(19)
memory usage: 1.2+ GB


Convert the contribution date and latest month columns to datetime data type.

In [3]:
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")
contributions["last_day"] = pd.to_datetime(contributions["last_day"], errors="coerce")
contributions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6947770 entries, 0 to 6947769
Data columns (total 23 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
contributor_street        object
contributor_city          object
contributor_state         object
contributor_zip           float64
in_out_state              object
no_veto                   object
office                    object
last_day                  datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(2), int64(2), object(17)
memory usage: 1.2+ GB


Filter the data by election cycle.

In [4]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_14 = contributions[contributions["year"] == 2014]
contributions_10 = contributions[contributions["year"] == 2010]

## Calculate out-of-state contributions by state and office

Group contributions by state, standardized office and in-vs.-out-of-state status and sum contributions.

In [5]:
contributions_by_state_18 = contributions_18.groupby(["state", "standardized_office", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_18.rename(columns={"amount": "contributions_18"}, inplace=True)
contributions_by_state_14 = contributions_14.groupby(["state", "standardized_office", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_14.rename(columns={"amount": "contributions_14"}, inplace=True)
contributions_by_state_10 = contributions_10.groupby(["state", "standardized_office", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_10.rename(columns={"amount": "contributions_10"}, inplace=True)
contributions_by_state_18.head()

,state,standardized_office,in_out_state,contributions_18
0,AK,GOVERNOR/LIEUTENANT GOVERNOR,in-state,"1,797,085.43"
1,AK,GOVERNOR/LIEUTENANT GOVERNOR,out-of-state,"93,260.00"
2,AK,GOVERNOR/LIEUTENANT GOVERNOR,unknown,308.86
3,AK,STATE HOUSE/ASSEMBLY/SENATE,in-state,"2,847,314.12"
4,AK,STATE HOUSE/ASSEMBLY/SENATE,out-of-state,"126,112.84"


Pivot dataframe to aggregate each state's data in a single row.

In [6]:
contributions_by_state_18 = pd.pivot_table(contributions_by_state_18, index=["state"], columns=["standardized_office", "in_out_state"]).reset_index()
contributions_by_state_14 = pd.pivot_table(contributions_by_state_14, index=["state"], columns=["standardized_office", "in_out_state"]).reset_index()
contributions_by_state_10 = pd.pivot_table(contributions_by_state_10, index=["state"], columns=["standardized_office", "in_out_state"]).reset_index()

It appears that some states have no contributions categorized as "unknown." Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [7]:
contributions_by_state_18.fillna(0, inplace=True)
contributions_by_state_14.fillna(0, inplace=True)
contributions_by_state_10.fillna(0, inplace=True)

Flatten the resulting dataframes' multi-index columns.

In [8]:
contributions_by_state_18.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_18.columns.values]
contributions_by_state_14.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_14.columns.values]
contributions_by_state_10.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_10.columns.values]

Calculate the total contributions by state and office.

In [9]:
# 2018
contributions_by_state_18["contributions_18_governor/lieutenant_governor_total"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"]
contributions_by_state_18["contributions_18_state_house/assembly/senate_total"] = contributions_by_state_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_unknown"]
# 2014
contributions_by_state_14["contributions_14_governor/lieutenant_governor_total"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"]
contributions_by_state_14["contributions_14_state_house/assembly/senate_total"] = contributions_by_state_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_unknown"]
# 2010
contributions_by_state_10["contributions_10_governor/lieutenant_governor_total"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"]
contributions_by_state_10["contributions_10_state_house/assembly/senate_total"] = contributions_by_state_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_unknown"]

Calculate the proportion of in-state, out-of-state and unknown contributions.

In [10]:
contributions_by_state_18["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] / (contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_18["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_18["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"] / (contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_18["pct_state_house/assembly/senate_in_state"] = contributions_by_state_18["contributions_18_state_house/assembly/senate_in_state"] / (contributions_by_state_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_unknown"])
contributions_by_state_18["pct_state_house/assembly/senate_out_of_state"] = contributions_by_state_18["contributions_18_state_house/assembly/senate_out_of_state"] / (contributions_by_state_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_unknown"])
contributions_by_state_18["pct_state_house/assembly/senate_unknown"] = contributions_by_state_18["contributions_18_state_house/assembly/senate_unknown"] / (contributions_by_state_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_unknown"])

In [11]:
cols = contributions_by_state_18.columns.tolist()
cols = cols[0:4] + cols[7:8] + cols[4:7] + cols[8:9] + cols[9:15]
contributions_by_state_18 = contributions_by_state_18[cols]
contributions_by_state_18

,state,contributions_18_governor/lieutenant_governor_in_state,contributions_18_governor/lieutenant_governor_out_of_state,contributions_18_governor/lieutenant_governor_unknown,contributions_18_governor/lieutenant_governor_total,contributions_18_state_house/assembly/senate_in_state,contributions_18_state_house/assembly/senate_out_of_state,contributions_18_state_house/assembly/senate_unknown,contributions_18_state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,"1,797,085.43","93,260.00",308.86,"1,890,654.29","2,847,314.12","126,112.84","3,885.00","2,977,311.96",0.95,0.05,0.00,0.96,0.04,0.00
1,AL,"14,897,487.29","975,117.14","722,137.62","16,594,742.05","20,981,082.71","1,470,891.02","-82,594.92","22,369,378.81",0.90,0.06,0.04,0.94,0.07,-0.00
2,AR,"4,020,644.69","485,939.52",0.00,"4,506,584.21","3,531,458.28","438,633.87","138,032.67","4,108,124.82",0.89,0.11,0.00,0.86,0.11,0.03
3,CA,"71,612,230.15","7,130,300.55","127,008.78","78,869,539.48","78,210,308.45","13,358,453.59","78,911.13","91,647,673.17",0.91,0.09,0.00,0.85,0.15,0.00
4,CO,"31,311,520.17","2,664,673.10","37,667.00","34,013,860.27","4,214,811.17","690,879.09","62,023.64","4,967,713.90",0.92,0.08,0.00,0.85,0.14,0.01
5,CT,"16,407,665.92","245,631.38","3,320,425.00","19,973,722.30","5,053,439.11","240,581.08","18,703.01","5,312,723.20",0.82,0.01,0.17,0.95,0.05,0.00
6,DE,0.00,0.00,0.00,0.00,"1,075,298.35","134,611.28","21,253.36","1,231,162.99",nan,nan,nan,0.87,0.11,0.02
7,FL,"103,558,702.01","5,611,560.90","117,035.20","109,287,298.11","35,852,941.94","3,798,559.92","2,394,075.76","42,045,577.62",0.95,0.05,0.00,0.85,0.09,0.06
8,GA,"32,804,615.27","5,322,211.80","13,850.00","38,140,677.07","16,088,266.12","3,222,579.45","354,491.98","19,665,337.55",0.86,0.14,0.00,0.82,0.16,0.02
9,HI,"5,688,173.91","852,563.36","1,200.00","6,541,937.27","3,287,952.55","416,525.76","280,829.56","3,985,307.87",0.87,0.13,0.00,0.83,0.10,0.07


In [12]:
contributions_by_state_14["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] / (contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_14["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_14["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"] / (contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_14["pct_state_house/assembly/senate_in_state"] = contributions_by_state_14["contributions_14_state_house/assembly/senate_in_state"] / (contributions_by_state_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_unknown"])
contributions_by_state_14["pct_state_house/assembly/senate_out_of_state"] = contributions_by_state_14["contributions_14_state_house/assembly/senate_out_of_state"] / (contributions_by_state_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_unknown"])
contributions_by_state_14["pct_state_house/assembly/senate_unknown"] = contributions_by_state_14["contributions_14_state_house/assembly/senate_unknown"] / (contributions_by_state_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_unknown"])

In [13]:
cols = contributions_by_state_14.columns.tolist()
cols = cols[0:4] + cols[7:8] + cols[4:7] + cols[8:9] + cols[9:15]
contributions_by_state_14 = contributions_by_state_14[cols]
contributions_by_state_14

,state,contributions_14_governor/lieutenant_governor_in_state,contributions_14_governor/lieutenant_governor_out_of_state,contributions_14_governor/lieutenant_governor_unknown,contributions_14_governor/lieutenant_governor_total,contributions_14_state_house/assembly/senate_in_state,contributions_14_state_house/assembly/senate_out_of_state,contributions_14_state_house/assembly/senate_unknown,contributions_14_state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,"2,134,137.09","140,361.26",200.00,"2,274,698.35","2,857,074.63","107,791.86",0.00,"2,964,866.49",0.94,0.06,0.00,0.96,0.04,0.00
1,AL,"5,634,820.31","722,814.00","8,000.00","6,365,634.31","26,035,311.46","1,507,166.67","20,102.53","27,562,580.66",0.89,0.11,0.00,0.94,0.05,0.00
2,AR,"6,647,148.58","913,024.35","17,050.45","7,577,223.38","3,752,761.36","492,279.25","89,261.51","4,334,302.12",0.88,0.12,0.00,0.87,0.11,0.02
3,CA,"25,686,985.90","3,451,797.30","10,300.00","29,149,083.20","84,586,169.16","11,070,018.96","27,732.83","95,683,920.95",0.88,0.12,0.00,0.88,0.12,0.00
4,CO,"7,241,563.12","1,405,940.61","6,812.68","8,654,316.41","6,146,809.16","434,044.59",563.99,"6,581,417.74",0.84,0.16,0.00,0.93,0.07,0.00
5,CT,"17,261,277.46","78,309.14",300.00,"17,339,886.60","7,957,301.86","112,233.38","4,221.82","8,073,757.06",1.00,0.00,0.00,0.99,0.01,0.00
6,DE,0.00,0.00,0.00,0.00,"1,728,996.08","318,521.50","16,493.19","2,064,010.77",nan,nan,nan,0.84,0.15,0.01
7,FL,"18,017,373.17","1,297,821.52","33,926.64","19,349,121.33","26,656,923.62","2,965,774.18","43,471.24","29,666,169.04",0.93,0.07,0.00,0.90,0.10,0.00
8,GA,"12,330,937.35","2,610,337.55",0.00,"14,941,274.90","12,659,238.40","2,780,518.08","1,000.00","15,440,756.48",0.83,0.17,0.00,0.82,0.18,0.00
9,HI,"6,355,665.68","1,286,113.73",0.00,"7,641,779.41","2,892,648.23","463,034.71",0.00,"3,355,682.94",0.83,0.17,0.00,0.86,0.14,0.00


In [14]:
contributions_by_state_10["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] / (contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_10["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_10["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"] / (contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_10["pct_state_house/assembly/senate_in_state"] = contributions_by_state_10["contributions_10_state_house/assembly/senate_in_state"] / (contributions_by_state_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_unknown"])
contributions_by_state_10["pct_state_house/assembly/senate_out_of_state"] = contributions_by_state_10["contributions_10_state_house/assembly/senate_out_of_state"] / (contributions_by_state_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_unknown"])
contributions_by_state_10["pct_state_house/assembly/senate_unknown"] = contributions_by_state_10["contributions_10_state_house/assembly/senate_unknown"] / (contributions_by_state_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_unknown"])

In [15]:
cols = contributions_by_state_10.columns.tolist()
cols = cols[0:4] + cols[7:8] + cols[4:7] + cols[8:9] + cols[9:15]
contributions_by_state_10 = contributions_by_state_10[cols]
contributions_by_state_10

,state,contributions_10_governor/lieutenant_governor_in_state,contributions_10_governor/lieutenant_governor_out_of_state,contributions_10_governor/lieutenant_governor_unknown,contributions_10_governor/lieutenant_governor_total,contributions_10_state_house/assembly/senate_in_state,contributions_10_state_house/assembly/senate_out_of_state,contributions_10_state_house/assembly/senate_unknown,contributions_10_state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,"2,748,199.13","160,882.45","2,336.62","2,911,418.20","2,359,834.42","76,247.50","4,700.73","2,440,782.65",0.94,0.06,0.00,0.97,0.03,0.00
1,AL,"23,762,864.10","1,110,388.41","16,430.55","24,889,683.06","21,500,923.89","1,087,955.58","181,979.54","22,770,859.01",0.95,0.04,0.00,0.94,0.05,0.01
2,AR,"1,856,752.48","91,000.00",300.00,"1,948,052.48","3,877,534.40","418,580.34","29,128.32","4,325,243.06",0.95,0.05,0.00,0.90,0.10,0.01
3,CA,"208,942,213.59","8,229,282.43","1,350.00","217,172,846.02","71,219,713.78","8,804,253.40","23,681.81","80,047,648.99",0.96,0.04,0.00,0.89,0.11,0.00
4,CO,"5,435,025.61","861,758.30",31.00,"6,296,814.91","5,704,169.53","578,079.70","-4,879.09","6,277,370.14",0.86,0.14,0.00,0.91,0.09,-0.00
5,CT,"26,636,208.59","681,321.42",0.00,"27,317,530.01","6,129,703.25","89,413.20",-15.00,"6,219,101.45",0.98,0.02,0.00,0.99,0.01,-0.00
6,DE,0.00,0.00,0.00,0.00,"1,777,871.86","279,038.61","8,634.64","2,065,545.11",nan,nan,nan,0.86,0.14,0.00
7,FL,"69,434,728.79","1,794,008.60","83,400.78","71,312,138.17","33,768,355.53","3,082,482.37","82,481.18","36,933,319.08",0.97,0.03,0.00,0.91,0.08,0.00
8,GA,"19,292,155.12","3,123,250.56",0.00,"22,415,405.68","11,132,885.99","1,983,911.57","1,925.00","13,118,722.56",0.86,0.14,0.00,0.85,0.15,0.00
9,HI,"2,632,205.28","508,933.00",0.00,"3,141,138.28","2,164,781.75","173,464.00",0.00,"2,338,245.75",0.84,0.16,0.00,0.93,0.07,0.00


### Reformat the data for maps

In [16]:
governors_map = contributions_by_state_18.iloc[:, [0, 2, 4, 10]]
governors_map

,state,contributions_18_governor/lieutenant_governor_out_of_state,contributions_18_governor/lieutenant_governor_total,pct_governor/lieutenant_governor_out_of_state
0,AK,"93,260.00","1,890,654.29",0.05
1,AL,"975,117.14","16,594,742.05",0.06
2,AR,"485,939.52","4,506,584.21",0.11
3,CA,"7,130,300.55","78,869,539.48",0.09
4,CO,"2,664,673.10","34,013,860.27",0.08
5,CT,"245,631.38","19,973,722.30",0.01
6,DE,0.00,0.00,nan
7,FL,"5,611,560.90","109,287,298.11",0.05
8,GA,"5,322,211.80","38,140,677.07",0.14
9,HI,"852,563.36","6,541,937.27",0.13


In [17]:
legislators_map = contributions_by_state_18.iloc[:, [0, 6, 8, 13]]
legislators_map

,state,contributions_18_state_house/assembly/senate_out_of_state,contributions_18_state_house/assembly/senate_total,pct_state_house/assembly/senate_out_of_state
0,AK,"126,112.84","2,977,311.96",0.04
1,AL,"1,470,891.02","22,369,378.81",0.07
2,AR,"438,633.87","4,108,124.82",0.11
3,CA,"13,358,453.59","91,647,673.17",0.15
4,CO,"690,879.09","4,967,713.90",0.14
5,CT,"240,581.08","5,312,723.20",0.05
6,DE,"134,611.28","1,231,162.99",0.11
7,FL,"3,798,559.92","42,045,577.62",0.09
8,GA,"3,222,579.45","19,665,337.55",0.16
9,HI,"416,525.76","3,985,307.87",0.10


## Calculate out-of-state contributions by state, office and party

Group contributions by state, standardized office, party and in-vs.-out-of-state status and sum contributions.

In [18]:
contributions_by_state_party_18 = contributions_18.groupby(["state", "standardized_office", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_party_18.rename(columns={"amount": "contributions_18"}, inplace=True)
contributions_by_state_party_14 = contributions_14.groupby(["state", "standardized_office", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_party_14.rename(columns={"amount": "contributions_14"}, inplace=True)
contributions_by_state_party_10 = contributions_10.groupby(["state", "standardized_office", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_party_10.rename(columns={"amount": "contributions_10"}, inplace=True)
contributions_by_state_party_18.head()

,state,standardized_office,party,in_out_state,contributions_18
0,AK,GOVERNOR/LIEUTENANT GOVERNOR,Democratic,in-state,"448,391.96"
1,AK,GOVERNOR/LIEUTENANT GOVERNOR,Democratic,out-of-state,"37,729.50"
2,AK,GOVERNOR/LIEUTENANT GOVERNOR,Democratic,unknown,250.00
3,AK,GOVERNOR/LIEUTENANT GOVERNOR,Republican,in-state,"841,763.22"
4,AK,GOVERNOR/LIEUTENANT GOVERNOR,Republican,out-of-state,"22,129.00"


Pivot dataframe to aggregate each state and party's data in a single row.

In [19]:
contributions_by_state_party_18 = pd.pivot_table(contributions_by_state_party_18, index=["state", "party"], columns=["standardized_office", "in_out_state"]).reset_index()
contributions_by_state_party_14 = pd.pivot_table(contributions_by_state_party_14, index=["state", "party"], columns=["standardized_office", "in_out_state"]).reset_index()
contributions_by_state_party_10 = pd.pivot_table(contributions_by_state_party_10, index=["state", "party"], columns=["standardized_office", "in_out_state"]).reset_index()

It appears that some states have no contributions categorized as "unknown." Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [20]:
contributions_by_state_party_18.fillna(0, inplace=True)
contributions_by_state_party_14.fillna(0, inplace=True)
contributions_by_state_party_10.fillna(0, inplace=True)

Flatten the resulting dataframes' multi-index columns.

In [21]:
contributions_by_state_party_18.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_party_18.columns.values]
contributions_by_state_party_14.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_party_14.columns.values]
contributions_by_state_party_10.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_party_10.columns.values]

Calculate the total contributions by state and office.

In [22]:
# 2018
contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_total"] = contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_unknown"]
contributions_by_state_party_18["contributions_18_state_house/assembly/senate_total"] = contributions_by_state_party_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_party_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_18["contributions_18_state_house/assembly/senate_unknown"]
# 2014
contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_total"] = contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_unknown"]
contributions_by_state_party_14["contributions_14_state_house/assembly/senate_total"] = contributions_by_state_party_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_party_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_14["contributions_14_state_house/assembly/senate_unknown"]
# 2010
contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_total"] = contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_unknown"]
contributions_by_state_party_10["contributions_10_state_house/assembly/senate_total"] = contributions_by_state_party_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_party_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_10["contributions_10_state_house/assembly/senate_unknown"]

Calculate the proportion of in-state, out-of-state and unknown contributions.

In [23]:
contributions_by_state_party_18["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_in_state"] / (contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_party_18["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_party_18["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_unknown"] / (contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_party_18["pct_state_house/assembly/senate_in_state"] = contributions_by_state_party_18["contributions_18_state_house/assembly/senate_in_state"] / (contributions_by_state_party_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_party_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_18["contributions_18_state_house/assembly/senate_unknown"])
contributions_by_state_party_18["pct_state_house/assembly/senate_out_of_state"] = contributions_by_state_party_18["contributions_18_state_house/assembly/senate_out_of_state"] / (contributions_by_state_party_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_party_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_18["contributions_18_state_house/assembly/senate_unknown"])
contributions_by_state_party_18["pct_state_house/assembly/senate_unknown"] = contributions_by_state_party_18["contributions_18_state_house/assembly/senate_unknown"] / (contributions_by_state_party_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_party_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_18["contributions_18_state_house/assembly/senate_unknown"])
contributions_by_state_party_18

,state,party,contributions_18_governor/lieutenant_governor_in_state,contributions_18_governor/lieutenant_governor_out_of_state,contributions_18_governor/lieutenant_governor_unknown,contributions_18_state_house/assembly/senate_in_state,contributions_18_state_house/assembly/senate_out_of_state,contributions_18_state_house/assembly/senate_unknown,contributions_18_governor/lieutenant_governor_total,contributions_18_state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,Democratic,"448,391.96","37,729.50",250.00,"1,323,760.05","59,793.09","3,880.00","486,371.46","1,387,433.14",0.92,0.08,0.00,0.95,0.04,0.00
1,AK,Republican,"841,763.22","22,129.00",58.86,"1,516,941.73","66,269.75",5.00,"863,951.08","1,583,216.48",0.97,0.03,0.00,0.96,0.04,0.00
2,AK,Third-Party,"506,930.25","33,401.50",0.00,"6,612.34",50.00,0.00,"540,331.75","6,662.34",0.94,0.06,0.00,0.99,0.01,0.00
3,AL,Democratic,"1,912,675.57","78,671.07","-16,485.22","3,898,401.69","341,336.42","-30,108.28","1,974,861.42","4,209,629.83",0.97,0.04,-0.01,0.93,0.08,-0.01
4,AL,Republican,"12,984,811.72","896,446.07","738,622.84","17,082,681.02","1,128,054.60","-52,486.64","14,619,880.63","18,158,248.98",0.89,0.06,0.05,0.94,0.06,-0.00
5,AL,Third-Party,0.00,0.00,0.00,0.00,"1,500.00",0.00,0.00,"1,500.00",nan,nan,nan,0.00,1.00,0.00
6,AR,Democratic,"206,496.47","84,632.52",0.00,"873,598.26","187,028.67","27,553.01","291,128.99","1,088,179.94",0.71,0.29,0.00,0.80,0.17,0.03
7,AR,Republican,"3,811,723.22","401,307.00",0.00,"2,640,815.02","251,355.20","110,479.66","4,213,030.22","3,002,649.88",0.90,0.10,0.00,0.88,0.08,0.04
8,AR,Third-Party,"2,425.00",0.00,0.00,"17,045.00",250.00,0.00,"2,425.00","17,295.00",1.00,0.00,0.00,0.99,0.01,0.00
9,CA,Democratic,"60,117,372.02","6,812,521.35","123,140.98","54,013,450.66","9,755,234.52","37,460.39","67,053,034.35","63,806,145.57",0.90,0.10,0.00,0.85,0.15,0.00


In [24]:
cols = contributions_by_state_party_18.columns.tolist()
cols = cols[0:5] + cols[8:9] + cols[5:8] + cols[9:10] + cols[10:16]
contributions_by_state_party_18 = contributions_by_state_party_18[cols]
contributions_by_state_party_18

,state,party,contributions_18_governor/lieutenant_governor_in_state,contributions_18_governor/lieutenant_governor_out_of_state,contributions_18_governor/lieutenant_governor_unknown,contributions_18_governor/lieutenant_governor_total,contributions_18_state_house/assembly/senate_in_state,contributions_18_state_house/assembly/senate_out_of_state,contributions_18_state_house/assembly/senate_unknown,contributions_18_state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,Democratic,"448,391.96","37,729.50",250.00,"486,371.46","1,323,760.05","59,793.09","3,880.00","1,387,433.14",0.92,0.08,0.00,0.95,0.04,0.00
1,AK,Republican,"841,763.22","22,129.00",58.86,"863,951.08","1,516,941.73","66,269.75",5.00,"1,583,216.48",0.97,0.03,0.00,0.96,0.04,0.00
2,AK,Third-Party,"506,930.25","33,401.50",0.00,"540,331.75","6,612.34",50.00,0.00,"6,662.34",0.94,0.06,0.00,0.99,0.01,0.00
3,AL,Democratic,"1,912,675.57","78,671.07","-16,485.22","1,974,861.42","3,898,401.69","341,336.42","-30,108.28","4,209,629.83",0.97,0.04,-0.01,0.93,0.08,-0.01
4,AL,Republican,"12,984,811.72","896,446.07","738,622.84","14,619,880.63","17,082,681.02","1,128,054.60","-52,486.64","18,158,248.98",0.89,0.06,0.05,0.94,0.06,-0.00
5,AL,Third-Party,0.00,0.00,0.00,0.00,0.00,"1,500.00",0.00,"1,500.00",nan,nan,nan,0.00,1.00,0.00
6,AR,Democratic,"206,496.47","84,632.52",0.00,"291,128.99","873,598.26","187,028.67","27,553.01","1,088,179.94",0.71,0.29,0.00,0.80,0.17,0.03
7,AR,Republican,"3,811,723.22","401,307.00",0.00,"4,213,030.22","2,640,815.02","251,355.20","110,479.66","3,002,649.88",0.90,0.10,0.00,0.88,0.08,0.04
8,AR,Third-Party,"2,425.00",0.00,0.00,"2,425.00","17,045.00",250.00,0.00,"17,295.00",1.00,0.00,0.00,0.99,0.01,0.00
9,CA,Democratic,"60,117,372.02","6,812,521.35","123,140.98","67,053,034.35","54,013,450.66","9,755,234.52","37,460.39","63,806,145.57",0.90,0.10,0.00,0.85,0.15,0.00


In [25]:
contributions_by_state_party_14["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_in_state"] / (contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_party_14["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_party_14["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_unknown"] / (contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_party_14["pct_state_house/assembly/senate_in_state"] = contributions_by_state_party_14["contributions_14_state_house/assembly/senate_in_state"] / (contributions_by_state_party_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_party_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_14["contributions_14_state_house/assembly/senate_unknown"])
contributions_by_state_party_14["pct_state_house/assembly/senate_out_of_state"] = contributions_by_state_party_14["contributions_14_state_house/assembly/senate_out_of_state"] / (contributions_by_state_party_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_party_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_14["contributions_14_state_house/assembly/senate_unknown"])
contributions_by_state_party_14["pct_state_house/assembly/senate_unknown"] = contributions_by_state_party_14["contributions_14_state_house/assembly/senate_unknown"] / (contributions_by_state_party_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_party_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_14["contributions_14_state_house/assembly/senate_unknown"])

In [26]:
cols = contributions_by_state_party_14.columns.tolist()
cols = cols[0:5] + cols[8:9] + cols[5:8] + cols[9:10] + cols[10:16]
contributions_by_state_party_14 = contributions_by_state_party_14[cols]
contributions_by_state_party_14

,state,party,contributions_14_governor/lieutenant_governor_in_state,contributions_14_governor/lieutenant_governor_out_of_state,contributions_14_governor/lieutenant_governor_unknown,contributions_14_governor/lieutenant_governor_total,contributions_14_state_house/assembly/senate_in_state,contributions_14_state_house/assembly/senate_out_of_state,contributions_14_state_house/assembly/senate_unknown,contributions_14_state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,Democratic,"755,837.42","64,737.07",200.00,"820,774.49","1,011,641.93","49,040.67",0.00,"1,060,682.60",0.92,0.08,0.00,0.95,0.05,0.00
1,AK,Republican,"899,769.23","49,058.00",0.00,"948,827.23","1,768,172.96","54,957.87",0.00,"1,823,130.83",0.95,0.05,0.00,0.97,0.03,0.00
2,AK,Third-Party,"478,530.44","26,566.19",0.00,"505,096.63","77,259.74","3,793.32",0.00,"81,053.06",0.95,0.05,0.00,0.95,0.05,0.00
3,AL,Democratic,"223,629.16","4,926.00",0.00,"228,555.16","4,844,291.69","377,401.14","5,602.53","5,227,295.36",0.98,0.02,0.00,0.93,0.07,0.00
4,AL,Republican,"5,411,191.15","717,888.00","8,000.00","6,137,079.15","20,947,911.63","1,124,615.53","14,500.00","22,087,027.16",0.88,0.12,0.00,0.95,0.05,0.00
5,AL,Third-Party,0.00,0.00,0.00,0.00,"243,108.14","5,150.00",0.00,"248,258.14",nan,nan,nan,0.98,0.02,0.00
6,AR,Democratic,"4,223,462.59","497,006.59","10,950.45","4,731,419.63","1,461,289.01","175,341.00","44,064.13","1,680,694.14",0.89,0.11,0.00,0.87,0.10,0.03
7,AR,Republican,"2,420,622.25","415,917.76","6,100.00","2,842,640.01","2,291,472.35","316,938.25","45,197.38","2,653,607.98",0.85,0.15,0.00,0.86,0.12,0.02
8,AR,Third-Party,"3,063.74",100.00,0.00,"3,163.74",0.00,0.00,0.00,0.00,0.97,0.03,0.00,nan,nan,nan
9,CA,Democratic,"19,727,676.63","2,839,973.92",0.00,"22,567,650.55","58,028,384.60","8,299,099.65","18,497.00","66,345,981.25",0.87,0.13,0.00,0.87,0.13,0.00


In [27]:
contributions_by_state_party_10["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_in_state"] / (contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_party_10["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_party_10["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_unknown"] / (contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_party_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_party_10["pct_state_house/assembly/senate_in_state"] = contributions_by_state_party_10["contributions_10_state_house/assembly/senate_in_state"] / (contributions_by_state_party_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_party_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_10["contributions_10_state_house/assembly/senate_unknown"])
contributions_by_state_party_10["pct_state_house/assembly/senate_out_of_state"] = contributions_by_state_party_10["contributions_10_state_house/assembly/senate_out_of_state"] / (contributions_by_state_party_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_party_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_10["contributions_10_state_house/assembly/senate_unknown"])
contributions_by_state_party_10["pct_state_house/assembly/senate_unknown"] = contributions_by_state_party_10["contributions_10_state_house/assembly/senate_unknown"] / (contributions_by_state_party_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_party_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_party_10["contributions_10_state_house/assembly/senate_unknown"])

In [28]:
cols = contributions_by_state_party_10.columns.tolist()
cols = cols[0:5] + cols[8:9] + cols[5:8] + cols[9:10] + cols[10:16]
contributions_by_state_party_10 = contributions_by_state_party_10[cols]
contributions_by_state_party_10

,state,party,contributions_10_governor/lieutenant_governor_in_state,contributions_10_governor/lieutenant_governor_out_of_state,contributions_10_governor/lieutenant_governor_unknown,contributions_10_governor/lieutenant_governor_total,contributions_10_state_house/assembly/senate_in_state,contributions_10_state_house/assembly/senate_out_of_state,contributions_10_state_house/assembly/senate_unknown,contributions_10_state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,Democratic,"697,266.31","44,396.15",-680.00,"740,982.46","1,073,603.01","38,075.00","2,007.10","1,113,685.11",0.94,0.06,-0.00,0.96,0.03,0.00
1,AK,Republican,"2,049,832.82","116,486.30","3,016.62","2,169,335.74","1,216,464.18","36,871.50","2,693.63","1,256,029.31",0.94,0.05,0.00,0.97,0.03,0.00
2,AK,Third-Party,"1,100.00",0.00,0.00,"1,100.00","69,767.23","1,301.00",0.00,"71,068.23",1.00,0.00,0.00,0.98,0.02,0.00
3,AL,Democratic,"4,964,170.53","424,689.98","2,750.00","5,391,610.51","11,224,174.80","484,472.37","171,501.60","11,880,148.77",0.92,0.08,0.00,0.94,0.04,0.01
4,AL,Republican,"18,798,693.57","685,698.43","13,680.55","19,498,072.55","9,740,984.24","526,383.21","22,756.76","10,290,124.21",0.96,0.04,0.00,0.95,0.05,0.00
5,AL,Third-Party,0.00,0.00,0.00,0.00,"535,764.85","77,100.00","-12,278.82","600,586.03",nan,nan,nan,0.89,0.13,-0.02
6,AR,Democratic,"1,800,194.16","85,500.00",300.00,"1,885,994.16","2,644,885.81","281,858.93","27,256.68","2,954,001.42",0.95,0.05,0.00,0.90,0.10,0.01
7,AR,Republican,"56,558.32","5,500.00",0.00,"62,058.32","1,229,967.34","136,721.41","1,871.64","1,368,560.39",0.91,0.09,0.00,0.90,0.10,0.00
8,AR,Third-Party,0.00,0.00,0.00,0.00,"2,681.25",0.00,0.00,"2,681.25",nan,nan,nan,1.00,0.00,0.00
9,CA,Democratic,"37,363,371.82","2,856,083.56",650.00,"40,220,105.38","50,012,458.16","7,038,565.32","7,050.00","57,058,073.48",0.93,0.07,0.00,0.88,0.12,0.00


## Export the data

In [29]:
writer = pd.ExcelWriter("data/states_analysis.xlsx")
contributions_by_state_18.to_excel(writer, "contributions_by_state_18", index=False)
contributions_by_state_14.to_excel(writer, "contributions_by_state_14", index=False)
contributions_by_state_10.to_excel(writer, "contributions_by_state_10", index=False)
contributions_by_state_party_18.to_excel(writer, "contributions_by_state_party_18", index=False)
contributions_by_state_party_14.to_excel(writer, "contributions_by_state_party_14", index=False)
contributions_by_state_party_10.to_excel(writer, "contributions_by_state_party_10", index=False)
writer.save()

In [30]:
governors_map.to_csv("data/governors_map.csv", index=False)
legislators_map.to_csv("data/legislators_map.csv", index=False)